# 성능튜닝

## 1.환경준비

### (1) import

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 853.4 kB/s eta 0:00:00


In [17]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import *

import warnings    # 경고메시지 제외
warnings.filterwarnings(action='ignore')

### (2) 데이터 준비

* 변수설명
    * COLLEGE : 대학 졸업여부
    * INCOME : 연수입
    * OVERAGE : 월평균 초과사용 시간(분)
    * LEFTOVER : 월평균 잔여시간비율(%)
    * HOUSE : 집값
    * HANDSET_PRICE : 스마트폰 가격
    * OVER_15MINS_CALLS_PER_MONTH : 월평균 장기통화(15분이상) 횟수
    * AVERAGE_CALL_DURATION : 평균 통화 시간
    * REPORTED_SATISFACTION : 만족도 설문조사 결과
    * REPORTED_USAGE_LEVEL : 사용도 자가진단 결과
    * CONSIDERING_CHANGE_OF_PLAN : 향후 변경계획 설문조사 결과
    * CHURN : 이탈(번호이동) 여부 (1-이탈, 0-잔류, Target 변수)


In [18]:
# 데이터를 불러옵시다.
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv'
data = pd.read_csv(path)
data = data.sample(5000, random_state = 2022)
data['CHURN'] = data['CHURN'].map({'LEAVE':1, 'STAY':0})
data.head()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
3178,3179,0,119512,51,31,248566,229,5,2,very_sat,very_high,considering,1
14926,14927,1,142144,192,15,774317,581,29,4,unsat,very_little,never_thought,1
15116,15117,1,142308,0,79,306426,497,1,1,sat,little,considering,0
12733,12734,1,113385,0,0,333599,819,1,6,very_unsat,very_high,considering,1
14032,14033,1,90348,209,10,637286,360,26,4,unsat,little,actively_looking_into_it,0


## 2.데이터 준비

### (1) 데이터 정리

In [19]:
drop_cols = ['id']
data.drop(drop_cols, axis = 1, inplace = True )

### (2) 데이터분할1 : x, y 나누기

In [20]:
target = 'CHURN'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### (3) NA 조치

### (4) 가변수화

In [21]:
dumm_cols = ['REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN']
x = pd.get_dummies(x, columns = dumm_cols, drop_first = True)

### (5) 데이터분할2 : train : validation 나누기

In [22]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (6) Scaling

In [23]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

## 3.선형모델 튜닝

Logistic Regression : 전진선택법
* 변수를 하나씩 늘려가면서
* AIC를 가장 낮추는 모델 찾기

### (1) 전진선택을 수행할 함수 만들기( **로지스틱 회귀** 용)

In [ ]:
# 아래 함수는 로지스틱 회귀를 위한 전진선택법 함수 입니다.
import statsmodels.api as sm

def forward_stepwise_logistic(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    #
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = sm.Logit(y_train, x_tr).fit()
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)

        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)

    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

### (2) 전진선택법 수행

In [ ]:
vars, result = forward_stepwise_logistic(x_train, y_train)

* 선택된 변수

In [ ]:
vars

['OVERAGE',
 'HOUSE',
 'HANDSET_PRICE',
 'LEFTOVER',
 'REPORTED_SATISFACTION_very_sat',
 'INCOME',
 'REPORTED_SATISFACTION_sat']

In [ ]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

### (3) 모델링

* 전체 변수

In [ ]:
m1 = LogisticRegression()
m1.fit(x_train, y_train)
p1 = m1.predict(x_val)

print(accuracy_score(y_val, p1))
print(classification_report(y_val, p1))

0.6333333333333333
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.65      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.63      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



* 전진선택법 변수

In [ ]:
m2 = LogisticRegression()
m2.fit(x_train[vars], y_train)
p2 = m2.predict(x_val[vars])

print(accuracy_score(y_val, p2))
print(classification_report(y_val, p2))

0.634
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.66      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.64      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



## 4.하이퍼파라미터 튜닝

### (1) 필요한 함수 불러오기

In [7]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### (2) Random Search

① 값의 범위를 지정한다.  
② 모델 선언(시도 횟수 지정)  
③ 모델링(값의 범위 내에서 시도 횟수만큼 랜덤하게 선택해서 시도한다.)  
④ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [ ]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(1,51), 'metric' : ['euclidean', 'manhattan']  }
params

{'metric': ['euclidean', 'manhattan'], 'n_neighbors': range(1, 51)}

#### ② 모델 선언

In [ ]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_rs = RandomizedSearchCV(model
                            , params              # hyperparameter 범위 지정.
                            , cv=5                    # k-fold Cross Validation
                            , n_iter=5                # Random하게 시도할 횟수
                            )

#### ③ 모델링

In [ ]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train_s, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'metric': ['euclidean', 'manhattan'],
                                        'n_neighbors': range(1, 51)})

In [ ]:
# 튜닝 결과
model_rs.cv_results_

{'mean_fit_time': array([0.00171256, 0.00172267, 0.00183244, 0.00175877, 0.00144153]),
 'mean_score_time': array([0.07686062, 0.0776505 , 0.08721075, 0.08160691, 0.07533393]),
 'mean_test_score': array([0.61771429, 0.58085714, 0.59457143, 0.598     , 0.618     ]),
 'param_metric': masked_array(data=['manhattan', 'euclidean', 'euclidean', 'euclidean',
                    'manhattan'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_neighbors': masked_array(data=[32, 15, 47, 49, 29],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'metric': 'manhattan', 'n_neighbors': 32},
  {'metric': 'euclidean', 'n_neighbors': 15},
  {'metric': 'euclidean', 'n_neighbors': 47},
  {'metric': 'euclidean', 'n_neighbors': 49},
  {'metric': 'manhattan', 'n_neighbors': 29}],
 'rank_test_score': array([2, 5, 4, 3, 1], dtype=int32),
 'split0_test_score': array([0.622857

In [ ]:
model_rs.cv_results_['params']

[{'metric': 'manhattan', 'n_neighbors': 32},
 {'metric': 'euclidean', 'n_neighbors': 15},
 {'metric': 'euclidean', 'n_neighbors': 47},
 {'metric': 'euclidean', 'n_neighbors': 49},
 {'metric': 'manhattan', 'n_neighbors': 29}]

In [ ]:
model_rs.cv_results_['mean_test_score']

array([0.61771429, 0.58085714, 0.59457143, 0.598     , 0.618     ])

In [ ]:
# 최적의 파라미터
model_rs.best_params_

{'metric': 'manhattan', 'n_neighbors': 29}

In [ ]:
# 그때의 성능
model_rs.best_score_

0.618

In [ ]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.59      0.72      0.65       738
           1       0.65      0.51      0.57       762

    accuracy                           0.61      1500
   macro avg       0.62      0.61      0.61      1500
weighted avg       0.62      0.61      0.61      1500



### (3) 실습 : Random Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [ ]:
# dictionary형태로 선언
params = { 'max_depth' : range(1,11), 'min_samples_leaf' : range(10,101,10)  }
params

{'max_depth': range(1, 11), 'min_samples_leaf': range(10, 101, 10)}

#### ② 모델 선언

In [ ]:
# 기본모델
model = DecisionTreeClassifier()

# Random Search 설정.
model_rs = RandomizedSearchCV(model
                            , params              # hyperparameter 범위 지정.
                            , cv = 5                    # k-fold Cross Validation
                            , n_iter = 10                # Random하게 시도할 횟수
                            )

#### ③ 모델링

In [ ]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(10, 101, 10)})

In [ ]:
# 튜닝 결과
model_rs.cv_results_

In [ ]:
model_rs.cv_results_['params']

[{'max_depth': 9, 'min_samples_leaf': 50},
 {'max_depth': 2, 'min_samples_leaf': 10},
 {'max_depth': 3, 'min_samples_leaf': 20},
 {'max_depth': 6, 'min_samples_leaf': 20},
 {'max_depth': 9, 'min_samples_leaf': 30},
 {'max_depth': 6, 'min_samples_leaf': 50},
 {'max_depth': 10, 'min_samples_leaf': 100},
 {'max_depth': 2, 'min_samples_leaf': 100},
 {'max_depth': 10, 'min_samples_leaf': 30},
 {'max_depth': 9, 'min_samples_leaf': 90}]

In [ ]:
model_rs.cv_results_['mean_test_score']

array([0.69114286, 0.66142857, 0.69314286, 0.68428571, 0.698     ,
       0.69114286, 0.69228571, 0.66142857, 0.69657143, 0.69542857])

In [ ]:
# 최적의 파라미터
model_rs.best_params_

{'max_depth': 9, 'min_samples_leaf': 30}

In [ ]:
# 그때의 성능
model_rs.best_score_

0.698

In [ ]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.67      0.68      0.68       738
           1       0.69      0.68      0.68       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500



### (4) Grid Search

① 값의 범위를 지정한다.  
② 모델링(값의 범위 내에서 모든 조합을 다 시도한다.)  
③ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [ ]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(3,31,2), 'metric' : ['euclidean', 'manhattan']  }
params

{'metric': ['euclidean', 'manhattan'], 'n_neighbors': range(3, 31, 2)}

#### ② 모델 선언

In [ ]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_gs = GridSearchCV(model, params, cv=5)

#### ③ 모델링

In [ ]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train_s, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(3, 31, 2)})

In [ ]:
# 튜닝 결과
model_gs.cv_results_

In [ ]:
model_gs.cv_results_['params']

In [ ]:
model_gs.cv_results_['mean_test_score']

array([0.57885714, 0.57942857, 0.57714286, 0.57342857, 0.568     ,
       0.57457143, 0.58085714, 0.58257143, 0.57914286, 0.58057143,
       0.586     , 0.58457143, 0.57971429, 0.58028571, 0.57314286,
       0.59142857, 0.59171429, 0.59142857, 0.59828571, 0.59771429,
       0.59942857, 0.60142857, 0.60171429, 0.61057143, 0.61      ,
       0.61142857, 0.61857143, 0.618     ])

In [ ]:
# 최적의 파라미터
model_gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 27}

In [ ]:
# 그때의 성능
model_gs.best_score_

0.6185714285714285

In [ ]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.59      0.72      0.65       738
           1       0.65      0.51      0.57       762

    accuracy                           0.61      1500
   macro avg       0.62      0.61      0.61      1500
weighted avg       0.62      0.61      0.61      1500



### (5) 실습 : Grid Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [ ]:
# dictionary형태로 선언
params = { 'max_depth' : range(2,11), 'min_samples_leaf' : range(10,101,20)  }
params

{'max_depth': range(2, 11), 'min_samples_leaf': range(10, 101, 20)}

#### ② 모델 선언

In [ ]:
# 기본모델
model = DecisionTreeClassifier()

# Random Search 설정.
model_gs = GridSearchCV(model, params, cv = 5)

#### ③ 모델링

In [ ]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(2, 11),
                         'min_samples_leaf': range(10, 101, 20)})

In [ ]:
# 튜닝 결과
model_gs.cv_results_

{'mean_fit_time': array([0.0073957 , 0.00615668, 0.00615282, 0.00611091, 0.00619969,
        0.00788407, 0.00992646, 0.00923018, 0.00817351, 0.00785065,
        0.00960717, 0.00967875, 0.00935097, 0.0093092 , 0.00972314,
        0.01123161, 0.01166892, 0.01114187, 0.01078081, 0.01082654,
        0.01325884, 0.01283312, 0.01195946, 0.01186709, 0.01068711,
        0.01428065, 0.01328917, 0.0122539 , 0.01162224, 0.01118779,
        0.01570706, 0.01411338, 0.01241632, 0.01177936, 0.01061616,
        0.01639433, 0.01494188, 0.01245885, 0.01120334, 0.0106781 ,
        0.01811824, 0.01414065, 0.01247907, 0.01122365, 0.01069455]),
 'mean_score_time': array([0.00180683, 0.00172501, 0.00169601, 0.0018095 , 0.00175009,
        0.00175138, 0.00228105, 0.0023623 , 0.00190411, 0.00178022,
        0.00187726, 0.00192618, 0.00176668, 0.00179524, 0.0021699 ,
        0.00190535, 0.00206518, 0.00195365, 0.00200319, 0.00207357,
        0.00202966, 0.00206499, 0.00214071, 0.00188651, 0.00195661,
        0.

In [ ]:
model_gs.cv_results_['params']

[{'max_depth': 2, 'min_samples_leaf': 10},
 {'max_depth': 2, 'min_samples_leaf': 30},
 {'max_depth': 2, 'min_samples_leaf': 50},
 {'max_depth': 2, 'min_samples_leaf': 70},
 {'max_depth': 2, 'min_samples_leaf': 90},
 {'max_depth': 3, 'min_samples_leaf': 10},
 {'max_depth': 3, 'min_samples_leaf': 30},
 {'max_depth': 3, 'min_samples_leaf': 50},
 {'max_depth': 3, 'min_samples_leaf': 70},
 {'max_depth': 3, 'min_samples_leaf': 90},
 {'max_depth': 4, 'min_samples_leaf': 10},
 {'max_depth': 4, 'min_samples_leaf': 30},
 {'max_depth': 4, 'min_samples_leaf': 50},
 {'max_depth': 4, 'min_samples_leaf': 70},
 {'max_depth': 4, 'min_samples_leaf': 90},
 {'max_depth': 5, 'min_samples_leaf': 10},
 {'max_depth': 5, 'min_samples_leaf': 30},
 {'max_depth': 5, 'min_samples_leaf': 50},
 {'max_depth': 5, 'min_samples_leaf': 70},
 {'max_depth': 5, 'min_samples_leaf': 90},
 {'max_depth': 6, 'min_samples_leaf': 10},
 {'max_depth': 6, 'min_samples_leaf': 30},
 {'max_depth': 6, 'min_samples_leaf': 50},
 {'max_dept

In [ ]:
model_gs.cv_results_['mean_test_score']

array([0.66142857, 0.66142857, 0.66142857, 0.66142857, 0.66142857,
       0.692     , 0.69314286, 0.69314286, 0.69314286, 0.69314286,
       0.68942857, 0.68942857, 0.68942857, 0.69285714, 0.69742857,
       0.686     , 0.69514286, 0.68857143, 0.69057143, 0.69771429,
       0.678     , 0.68714286, 0.69114286, 0.69142857, 0.69571429,
       0.67428571, 0.70057143, 0.68971429, 0.69      , 0.69542857,
       0.66742857, 0.694     , 0.69228571, 0.69      , 0.69542857,
       0.66457143, 0.698     , 0.69114286, 0.69      , 0.69542857,
       0.65685714, 0.69657143, 0.69      , 0.69      , 0.69542857])

In [ ]:
# 최적의 파라미터
model_gs.best_params_

{'max_depth': 7, 'min_samples_leaf': 30}

In [ ]:
# 그때의 성능
model_rs.best_score_

0.698

In [ ]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.66      0.70      0.68       738
           1       0.69      0.66      0.68       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500



## 5.베이지안 최적화

* 시간이 오래 걸리는 모델링에 적절
* 절차
    * 초기화: 평가 함수를 무작위로 몇 번 평가하여 초기 데이터를 수집합니다.
    * 대리 모델 학습: 수집된 데이터를 사용하여 평가 함수의 대리 모델을 학습시킵니다.
    * 획득 함수 최적화: 현재 대리 모델을 기반으로 획득 함수를 최대화하는 지점을 찾습니다. 이 지점은 다음에 평가할 가장 유망한 지점으로 간주됩니다.
    * 평가 함수 평가: 획득 함수에 의해 선택된 지점에서 평가 함수를 평가합니다.
    * 데이터 업데이트: 새로운 평가 결과를 기존 데이터에 추가합니다.
    * 반복: 대리 모델을 업데이트하고, 3~5단계를 최적의 솔루션이 발견되거나, 반복 횟수가 사전에 정한 한도에 도달할 때까지 반복합니다.

In [10]:
from skopt import BayesSearchCV
from skopt.space import Integer

In [14]:
# skopt의 Integer 함수 사용
params = {
    'max_depth': Integer(2, 10),
    'min_samples_leaf': Integer(10, 100)
}

In [24]:
# BayesSearchCV 설정
model_bs = BayesSearchCV(DecisionTreeClassifier(), params, n_iter=10, random_state=0)

# 최적화 실행
model_bs.fit(x_train, y_train)

# 최적의 하이퍼파라미터와 점수 출력
print("Best parameters found: ", model_bs.best_params_)
print("Best score found: ", model_bs.best_score_)

Best parameters found:  OrderedDict([('max_depth', 4), ('min_samples_leaf', 98)])
Best score found:  0.6934285714285715
